In [1]:
# Import required libraries
import wget
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [2]:
#Downloading the skeleton version of the dash app
#wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py")

In [3]:
# Downloading the dataset
#wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

In [4]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("Datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [5]:
spacex_df['Launch Site'].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [6]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [7]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Record Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36', 'text-transform': 'uppercase', 'font-family': 'Helvetica',
                                               'font-size': 40, 'font-style': 'bold', 'padding': '.1em 3em 0em 3em', 'background-color' : 'white'}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown', 
                                             options=[{'label': 'All Sites', 'value': 'ALL'},
                                                      {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                      {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                      {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                      {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                            value = 'Selected Launch Site',
                                             placeholder= 'Select Launch Site',
                                             searchable = True,
                                             style={'textAlign': 'left', 'color': 'black',
                                               'font-size': 20, 'font-': 'bold', 'width': '50%', 'padding': '0em 2em 0em .2em', 'background-color' : 'white', 
                                               'font-family': 'Helvetica', 'text-decoration': 'bold', 'text-transform': 'uppercase'}),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart'), style={'color': '#503D36', 'text-transform': 'uppercase',
                                               'font-size': 20, 'text-decoration': 'bold', 'padding': '0em 2em 0em 2em', 'background-color' : 'white'}),

                                html.P("Payload range (Kg):", style={'textAlign': 'left', 'color': 'black', 'border': 'none',
                                               'font-size': 20, 'font-style': 'bold', 'padding': '0em 8em 0em 0em', 'background-color' : 'white', 
                                               'font-family': 'Helvetica'}),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000, step= 1000,
                                                marks={0: '0', 500: '500', 1000: '1000', 1500: '1500', 2000: '2000', 2500: '2500', 3000: '3000', 3500: '3500', 4000: '4000', 4500: '4500', 5000: '5000', 
                                                       5500: '5500', 6000: '6000', 6500: '6500', 7000: '7000', 7500: '7500', 8000: '8000', 8500: '8500', 9000: '9000', 9500: '9500', 10000: '10000'}, 
                                                value= [min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart'),  style={'color': '#503D36', 'text-transform': 'uppercase',
                                               'font-size': 20, 'text-decoration': 'bold', 'padding': '0em 2em 0em 2em', 'background-color' : 'white'}),
                                ], style={'color': '#503D36', 'text-transform': 'uppercase',
                                               'font-size': 25, 'text-decoration': 'bold', 'padding': '0em 5em', 'background-color' : 'white'})

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value')
              )
def get_pie_chart(site_input):
    filtered_df = spacex_df
    if (site_input == 'ALL'):
        fig = px.pie(filtered_df, values='class',
                     names='Launch Site',
                     title='Total Success Launches By All Sites')
        return fig
    else:
        fig = px.pie(filtered_df, values=filtered_df['class'],
                    names=filtered_df['Launch Site'] == site_input,
                    title= 'Total Successful Launches For Site ' + site_input)
        return fig
    
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value'))
def get_scatter_chart(site_input, payload_slider):
    unfiltered_df = spacex_df
    low, high = payload_slider
    mask = (unfiltered_df['Payload Mass (kg)'] > low) & (unfiltered_df['Payload Mass (kg)'] < high)
    if (site_input == 'ALL'):
        fig = px.scatter(data_frame=unfiltered_df[mask], x='Payload Mass (kg)', y='class',
                         size='Payload Mass (kg)', hover_data='Payload Mass (kg)', color='Booster Version Category',
                     title='Relationship Between Payload Mass (kg) And Successful Launches For By All Sites')
        return fig
    else:
        filtered_df = spacex_df.loc[spacex_df['Launch Site'] == site_input]
        fig = px.scatter(data_frame=filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category', 
                         size='Payload Mass (kg)', hover_data='Payload Mass (kg)', 
                         title= 'Relationship Between Payload Mass (kg) & Successful Launches For Site ' + site_input)
        return fig
        

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output


# Run the app
if __name__ == '__main__':
    app.run_server()
